In [ ]:
import numpy as np
import math
import os 
import random
import argparse
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import cdist

parser = argparse.ArgumentParser()
parser.add_argument("--verbosity", help="increase output verbosity")
parser.add_argument('--input_type', type=str, default='pn',
                    help='type of input file, cif|pdb')
args = parser.parse_args(args=[])


#读取Protein-Net坐标
def atom_net(coord,atom_id):
    atom = np.array([float(coord[0].split()[atom_id])] + [float(coord[1].split()[atom_id])]+ [float(coord[2].split()[atom_id])])/100
    return atom


#读取cif坐标
def atom_cif(atoms, id_):
    atom = np.array([float(atoms[id_].split()[10])] + [float(atoms[id_].split()[11])] + [float(atoms[id_].split()[12])])
    return atom


#根据两点求单位向量
def vector_unit(vector_1,vector_2):
    bond_vector_2 = vector_1 - vector_2 
    bond_length_2 = np.linalg.norm(bond_vector_2)
    return bond_vector_2 / bond_length_2


#计算标准法向量 
def normal_vector_(B, C, D):
    U_1 = vector_unit(C,B) ; U = vector_unit(D,B)    
    N   = np.cross(U_1,U) / np.linalg.norm(np.cross(U_1,U)) 
    return  N  


#根据矩阵坐标求距离矩阵 
def contact_martix(A):
    # A是一个向量矩阵：euclidean代表欧式距离
    distA=pdist(A,metric='euclidean')
    # 将distA数组变成一个矩阵
    distB = squareform(distA)
    return distB


#计算旋转角和坐标转换权重
def torsion_angle(A, B, C, D):
    #计算法向量
    U_2 = vector_unit(B,A) ; U_1 = vector_unit(C,B) ; U = vector_unit(D,C)    
    N   = np.cross(U_1,U)   / np.linalg.norm(np.cross(U_1,U)) 
    N_1 = np.cross(U_2,U_1) / np.linalg.norm(np.cross(U_2,U_1))
    m_weight = np.array([U_1 , np.cross(N_1,U_1) , N_1]) 
    #torsion_angle
    angle = np.sign(np.dot(U_2,N)) * math.acos(np.dot(N_1,N))#+np.random.normal(loc=0.0, scale=1, size=None)*math.pi/18
    return angle, m_weight


#根据真实角度或训练角度预测下一个坐标
def next_coord(A, B, C, D, R, angle_confirm, angle_train):
    #torsion_angle
    angle_real , m = torsion_angle(A, B, C, D)
    #将真实角度或预测角度赋值给torsion
    torsion = angle_real
#     print("N——angle:",angle_real,angle_train)
    angle_martix=[math.cos(math.pi-angle_confirm),
                  math.sin(math.pi-angle_confirm) * math.cos(torsion),
                  math.sin(math.pi-angle_confirm) * math.sin(torsion)]
    #计算下一个坐标
    next_corrd = C + R * np.dot(m.T, angle_martix)
    return next_corrd, torsion


#计算旋转角和坐标转换权重
def torsion_angle_C(A, B, C, D):
    #计算法向量
    U_2 = vector_unit(B,A) ; U_1 = vector_unit(C,B) ; U = vector_unit(D,B)    
    N   = np.cross(U_1,U)   / np.linalg.norm(np.cross(U_1,U)) 
    N_1 = np.cross(U_2,U_1) / np.linalg.norm(np.cross(U_2,U_1))
    m_weight = np.array([U_1 , np.cross(N_1,U_1) , N_1]) 
    #torsion_angle
    angle = np.sign(np.dot(U_2,N)) * math.acos(np.dot(N_1,N))#+np.random.normal(loc=0.0, scale=1, size=None)*math.pi/18
    return angle, m_weight


#根据真实角度或训练角度 沿CA-CA轴旋转 预测同一平面的C
def next_coord_C(A, B, C, D, R, angle_confirm , angle_train):
    #torsion_angle
    angle_real , m = torsion_angle_C(A, B, C, D)
    #将真实角度或预测角度赋值给torsion
    torsion = angle_real
#     print("C——angle:",angle_real,angle_train)
    angle_martix=[math.cos(math.pi-angle_confirm),
                  math.sin(math.pi-angle_confirm) * math.cos(torsion),
                  math.sin(math.pi-angle_confirm) * math.sin(torsion)]
    #计算下一个坐标
    next_corrd = B + R * np.dot(m.T, angle_martix)
    return next_corrd, torsion


#根据真实坐标计算旋转角
def Cartesian_to_angle(path_file):
    if args.input_type == 'cif':
        f = open(path_file, 'r') ; lines = f.readlines() ; atoms = len(lines)
    else:
        f     = open(path_file, 'r')   ;   f_line = f.readlines()
        xyz   = f_line[-6:-3]          ;   chain  = f_line[-2]
        atoms = chain.count('+') * 3   ;   n_zero = chain[0:len(chain)//2].count('-') * 3; 
        
    for i in range(2,atoms-1):
        #读取数据
        if args.input_type == 'cif':
            A = atom_cif(lines, i-2)    ; B = atom_cif(lines, i-1)     ; C = atom_cif(lines, i)      ; D = atom_cif(lines, i+1)
        else:
            atom_id = n_zero+i  
            A = atom_net(xyz, atom_id-2); B = atom_net(xyz, atom_id-1) ; C = atom_net(xyz, atom_id)  ; D = atom_net(xyz, atom_id+1)
        #计算单位向量
        U_2 = vector_unit(B,A) ; U_1 = vector_unit(C,B) ; U = vector_unit(D,C)  
        #计算法向量
        N = np.cross(U_1,U) / np.linalg.norm(np.cross(U_1,U)) ; N_1 = np.cross(U_2,U_1) / np.linalg.norm(np.cross(U_2,U_1))
        #计算角度
        angle = np.sign(np.dot(U_2,N)) * math.acos(np.dot(N_1,N))/math.pi * 180 ; print(angle)
    f.close() 
          
        
#复现05-文献 基于cif格式文件  
def angle_to_Cartesian_cif(path, path_angle):
    f=open(path,'r') ; lines=f.readlines() ; total = 0 ; distance = 0
    A = atom_cif(lines, 0)   ; B = atom_cif(lines, 1)  ; C = atom_cif(lines, 2) 
    #构建接触矩阵
    true = np.zeros(shape=(len(lines),3))      
    true[0] =  A       ;  true[1] =  B       ; true[2] =  C ;
    
    generation = np.zeros(shape=(len(lines),3))
    generation[0] =  A ;  generation[1] =  B ; generation[2] =  C
    
    #读取预测的角度信息
    if not os.path.exists(path_angle):
        torsion_training = 0
    else:
        torsion_training = np.load(path_angle)
        
    #计算下一个原子坐标
    for i in range(2,len(lines)-1):
            #给定键长键角
        if lines[i].split()[3]   == 'CA':
            R = 1.52326  ; angle_confirm = 1.941
        elif lines[i].split()[3] == 'C':
            R = 1.32868; angle_confirm= 2.028
        elif lines[i].split()[3] == 'N':
            R = 1.45801 ; angle_confirm = 2.124 
         
        #获取当前原子的预测旋转角     
        torsion_train   = torsion_training[i//3]  
        
        D = atom_cif(lines, i+1) ; next_xyz, angle = next_coord(A, B, C, D, R, angle_confirm, torsion_training)
        true[i+1] = D            ; generation[i+1] = next_xyz
        total += np.square(np.linalg.norm(next_xyz - D))
        A = B ; B = C ; C = next_xyz
    
    #根据接触矩阵计算rmsd 
    T = contact_martix(true) ; G = contact_martix(generation)
    distance = np.square(np.linalg.norm(T - G))
    rmsd = np.sqrt(distance/(len(lines)-1)/len(lines))
    print(rmsd,len(lines))
    #由于误差会传递，所以rmsd较大
    
    #求接触矩阵的差的接触矩阵
#     dist=cdist(true,generation,metric='euclidean')
# #     print(dist)
#     rmsd = np.square(np.linalg.norm(dist))/len(lines)/(len(lines)-1)
#     print(rmsd,len(lines)) ; f.close()、
    f.close()

    
#复现05-文献 基于Protein-Net
def angle_to_Cartesian(path_file, path_angle):
    total = 0                      ;   distance = 0
    f     = open(path_file, 'r')   ;   f_line = f.readlines()
    xyz   = f_line[-6:-3]          ;   chain  = f_line[-2]
    atoms = chain.count('+') * 3   ;   n_zero = chain[0:len(chain)//2].count('-') * 3; 
    # Constants
    A = atom_net(xyz, n_zero + 0)         ; B = atom_net(xyz, n_zero + 1)            ; C = atom_net(xyz, n_zero + 2) 
    true = np.zeros(shape=(atoms,3))      ; true[0] =  A       ;  true[1] =  B       ; true[2] =  C ;
    generation = np.zeros(shape=(atoms,3)); generation[0] =  A ;  generation[1] =  B ; generation[2] =  C
        
    #读取预测的角度信息
    if not os.path.exists(path_angle):
        torsion_training = 0
    else:
        torsion_training = np.load(path_angle)
        
    #计算下一个原子坐标
    for i in range(2,atoms-1):
        D = atom_net(xyz, atom_id+1)  
        atom_id = n_zero+i  
        
        #给定键长键角
        if   atom_id % 3 == 1 :
            R = 1.52326  ; angle_confirm = 1.941
        elif atom_id % 3 == 2 :
            R = 1.32868; angle_confirm= 2.028
        elif atom_id % 3 == 0:
            R = 1.45801 ; angle_confirm = 2.124 
            
        #获取当前原子的预测旋转角     
        torsion_train   = torsion_training[i//3]  
        #预测下一个原子坐标
        next_xyz, angle = next_coord(A, B, C, D, R, angle_confirm, torsion_training)
        
        #构建接触矩阵
        true[i+1]       = D       
        generation[i+1] = next_xyz
        total += np.square(np.linalg.norm(next_xyz - D))
        A = B ; B = C ; C = next_xyz
        
    #根据接触矩阵计算rmsd 
    T = contact_martix(true) ; G = contact_martix(generation)
    distance = np.square(np.linalg.norm(T - G))
    rmsd = np.sqrt(distance/(atoms-1)/atoms)
    print(rmsd,atoms)
    #由于误差会传递，所以rmsd较大
    f.close()
    
    
#以CA-CA轴为旋转轴 根据旋转角预测原子坐标 （三种情况判断误差最小的情况）
def angle_to_Cartesian_CA_compare(path_file,path_angle):
    total = 0 ;  distance = 0; a=0; b=0; c=0
    
    if args.input_type == 'cif':
        f = open(path_file, 'r');   lines  = f.readlines();  atoms = len(lines)
        next_N = atom_cif(lines,0)
        
    else:
        f = open(path_file, 'r');            f_line = f.readlines()
        xyz = f_line[-6:-3];                 chain  = f_line[-2]
        atoms = chain.count('+') * 3;        n_zero = chain[0:len(chain)//2].count('-') * 3;
        
    #读取预测的角度信息
    if not os.path.exists(path_angle):
        torsion_training = 0
    else:
        torsion_training = np.load(path_angle)
        
    for i in range(2,atoms-4,3):         
        if args.input_type == 'cif':
            A = atom_cif(lines,i-2);   B = atom_cif(lines,i-1);     C = atom_cif(lines,i+2)
            D_C = atom_cif(lines, i);  D_N = atom_cif(lines, i+1);  C_2 = atom_cif(lines,i+3)      
            
        else:
            atom_id = n_zero+i 
            A = atom_net(xyz, atom_id-2); B = atom_net(xyz, atom_id-1);    C   = atom_net(xyz, atom_id + 2)  
            D_C = atom_net(xyz, atom_id) ; D_N = atom_net(xyz, atom_id+1);  C_2 = atom_net(xyz, atom_id+3)  
            
        #获取当前原子的预测旋转角     
        torsion_train = torsion_training[i//3]     
        
        #CA_next - C沿CA - CA轴做旋转
        #0.21941264623804932：C-CA轴和CA-CA轴的夹角
        R_C = 2.4345193937977068  ; angle_confirm_C = 0.21941264623804932
        next_C , angle_C = next_coord(A, B, C, D_C, R_C, angle_confirm_C, torsion_training)
        
        #CA - C沿CA - CA轴做旋转
        #0.35529281510453287：CA-C轴和CA-CA轴的夹角
        R_C_2 = 1.52326            ; angle_confirm_C_2 = math.pi -  0.35529281510453287
        next_C_2 , angle_C_2 = next_coord_C(A, B, C, D_C, R_C_2, angle_confirm_C_2, torsion_training)
        
        #CA_next - N沿CA - CA轴做旋转
        #0.263502970667963：CA-N轴和CA-CA轴的夹角
        R_N = 1.45801              ; angle_confirm_N =  0.263502970667963
        next_N , angle_N = next_coord(A, B, C, D_N, R_N, angle_confirm_N, torsion_training)
        
        total_C = np.linalg.norm(next_C - D_C)
        total_C_2 = np.linalg.norm(next_C_2 - D_C)
        total_N = np.linalg.norm(next_N - D_N)
        a += total_C;  b += total_C_2;  c += total_N
        x=(atoms - 6) // 3 
        print(a/x, b/x, c/x)
        f.close()

    
def angle_to_Cartesian_CA_CA(path_file,path_angle):
    total = 0;   distance = 0;  a = 0;   b = 0
    #读取数据
    if args.input_type == 'cif':
        f = open(path_file, 'r');   lines  = f.readlines();  atoms = len(lines)
        next_N = atom_cif(lines,0)
        
        #建造接触矩阵
        true = np.zeros(shape=(atoms,3));    generation = np.zeros(shape=(atoms,3))
        true[0]   = atom_cif(lines,0);       generation[0] = atom_cif(lines,0)
        true[-1]  = atom_cif(lines,atoms-1); generation[-1] = atom_cif(lines,atoms-1)
        
    else:
        f = open(path_file, 'r');            f_line = f.readlines()
        xyz = f_line[-6:-3];                 chain  = f_line[-2]
        atoms = chain.count('+') * 3;        n_zero = chain[0:len(chain)//2].count('-') * 3;
        
        #建造接触矩阵
        true = np.zeros(shape=(atoms,3));    generation = np.zeros(shape=(atoms,3))
        true[0] = atom_net(xyz,0);           generation[0] = atom_net(xyz,0)
        true[-1] = atom_net(xyz,atoms-1);    generation[-1] = atom_net(xyz,atoms-1)
        
    #读取预测的角度信息
    torsion_training = np.zeros(shape=(len(torsion_sin),1))
    if not os.path.exists(path_angle):
        torsion_training = np.zeros(shape=(len(torsion_sin),1))
        torsion_training[0] = 'none'
    else:
        torsion_sin = np.load(path_angle)[0] 
        torsion_cos = np.load(path_angle)[1] 
    #         torsion_training = np.load(path_angle)[2] * math.pi
        torsion_training = np.zeros(shape=(len(torsion_sin),1))
    
    for n in range(len(torsion_sin)):
        torsion_training[n] = math.atan2(torsion_sin[n],torsion_cos[n])
    
    for i in range(5,atoms-1,3):
        
        if args.input_type == 'cif':
            A = atom_cif(lines, i-4);      B = atom_cif(lines,i-1);       C = atom_cif(lines,i+2)
            D_C = atom_cif(lines, i);      D_N = atom_cif(lines, i+1) 
        else:
            atom_id = n_zero+i 
            A = atom_net(xyz, atom_id-4);  B = atom_net(xyz, atom_id-1);  C = atom_net(xyz, atom_id + 2)  
            D_C = atom_net(xyz, atom_id);  D_N = atom_net(xyz, atom_id+1)
        
        if torsion_training[0] != 'none':
            torsion_train_N = torsion_training[i//3]   #介入训练的角度

            if torsion_train_N >0:
                torsion_train_C = torsion_train_N - math.pi 
            else:
                torsion_train_C = math.pi + torsion_train_N
                
        #CA - C沿CA - CA轴做旋转    
        #0.35529281510453287：CA-C轴和CA-CA轴的夹角
        R_C = 1.52326          
        angle_confirm_C = math.pi -  0.35529281510453287
        next_C , angle_C = next_coord_C(A, B, C, D_C, R_C, angle_confirm_C, torsion_train_C)
        
        #CA_next - N沿CA - CA轴做旋转
        #0.263502970667963：CA-N轴和CA-CA轴的夹角
        R_N = 1.45801
        angle_confirm_N =  0.263502970667963
        next_N , angle_N =  next_coord (A, B, C, D_N, R_N, angle_confirm_N, torsion_train_N)
        
        true[i-1] = B;   generation[i-1] = B
        true[i]   = D_C; generation[i]   = next_C
        true[i+1] = D_N; generation[i+1] = next_N
#         print(next_C-D_C)

    #构建接触矩阵，计算rmsd     
    T = contact_martix(true) ; G = contact_martix(generation)
    distance = np.square(np.linalg.norm(T - G))
    rmsd = np.sqrt(distance/((atoms-1)*atoms))
    print(rmsd,atoms)
    
    
    #另一种度量损失方式以及建立文档记录的代码
#     write_rmsd = ('ProteinNet-文件名:'+ path_file.split('\\')[-1] +'\n' + 
#                   '接触矩阵的rmsd: ' + str(rmsd) + '\n------------------------------------------')
#     total_C = np.square(np.linalg.norm(next_C - D_C))
#     total_N = np.square(np.linalg.norm(next_N - D_N))
#     a += total_C  ; b += total_N

#     x=(atoms-6)//6 ;  write_rmsd = ('ProteinNet-文件名:'+path_file.split('\\')[-1]+'\n'+'  C原子的平均误差:'+str(np.sqrt(a/(x)))+'\n'
#                             '  N原子的平均误差:'+ str(np.sqrt(b/(x)))+'\n------------------------------------------')
#     print(write_rmsd)
#     f_rmsd.write(write_rmsd +'\n')
#     print(np.linalg.norm(next_N-next_C) )
#     print(angle_C,angle_N) ;
    f.close()
    

def angle_to_Cartesian_intersection(path_file,path_angle):
    total = 0;   distance = 0;  a = 0;   b = 0
    #读取数据
    if args.input_type == 'cif':
        f = open(path_file, 'r');   lines  = f.readlines();  atoms = len(lines)
        next_N = atom_cif(lines,0)
        
    else:
        f = open(path_file, 'r');            f_line = f.readlines()
        xyz = f_line[-6:-3];                 chain  = f_line[-2]
        atoms = chain.count('+') * 3;        n_zero = chain[0:len(chain)//2].count('-') * 3;

    for i in range(5,atoms-1,3):
        
        if args.input_type == 'cif':
            A   = atom_cif(lines, i-4);    B   = atom_cif(lines,i-1);       C = atom_cif(lines,i+2)
            D_C = atom_cif(lines, i);      D_N = atom_cif(lines, i+1);      N = atom_cif(lines,i-2)
        else:
            atom_id = n_zero+i 
            A = atom_net(xyz, atom_id-4);  B   = atom_net(xyz, atom_id-1);    C = atom_net(xyz, atom_id + 2)  
            D_C = atom_net(xyz, atom_id);  D_N = atom_net(xyz, atom_id+1) ;   N = atom_net(xyz, atom_id-2);
         
        #根据键长键角计算圆环的半径
        #0.35529281510453287：CA-C轴和CA-CA轴的夹角
        R_C = 1.52326 * math.cos(0.35529281510453287)
        R_C_2 = 1.52326 * math.cos(math.pi - 1.941)
        
        #求第一个法向量    CA-CA轴
        normal_vector_1 =  (C-B)/np.linalg.norm(C-B)
        #求第一个圆心    CA-CA轴
        next_CA_mid = B +  R_C * normal_vector_1
        
        #求第二个法向量    C-N轴
        normal_vector_2 =  (B-N)/np.linalg.norm(B-N)
        #求第二个圆心    CA-CA轴
        next_C_mid =  B +  R_C_2 * normal_vector_2
        #两个法向量之间的角度不是固定的
        
#         print(np.dot(normal_vector_2,vector_unit(next_C_2,next_C_mid)))#验证法向量
        #半径大的误差大
    
        #根据求出的半径和圆心复原坐标 ，验证半径和圆心是否计算正确 是否可还原坐标
        true_C = next_C_mid + ((D_C - next_C_mid) / np.linalg.norm(D_C - next_C_mid)) * 1.52326 * math.sin(math.pi - 1.941)
        true_C_1 = next_CA_mid + ((D_C - next_CA_mid) / np.linalg.norm(D_C - next_CA_mid)) * 0.5298886988235514
        #0.5298886988235514：CA-C轴绕CA-CA轴旋转的半径
        
        
        #两平面相交直线L0的方向向量
        L0_dir = np.cross(normal_vector_1,normal_vector_2) / np.linalg.norm(np.cross(normal_vector_1,normal_vector_2)) 
        
        #平面1上于L0垂直的L1的方向向量
        L1_dir = np.cross(L0_dir,normal_vector_1) / np.linalg.norm(np.cross(L0_dir,normal_vector_1)) 
        
        #两圆心连起来的的向量ps
        ps = next_C_mid - next_CA_mid #p-s     
        
        #计算平面1的圆心到平面2的距离D
        D = np.dot(ps,normal_vector_2)
        
        #计算平面1圆心到交线L0的距离
        cos_a_n = np.dot(L1_dir,normal_vector_2)
        t = D / cos_a_n
        
        #求出L1和L0的交点
        R = next_CA_mid + t* L1_dir
        print(D,t)
        
        #从两个交点中找出和正确坐标更近的点
        L0  = R - np.sqrt((0.5298886988235514**2)-t**2) * L0_dir
        L0_ = R + np.sqrt((0.5298886988235514**2)-t**2) * L0_dir
        
        print(np.linalg.norm(L0 - next_CA_mid))
        print(min(np.linalg.norm(L0 - D_C),np.linalg.norm(L0_ - D_C)))

        print('--------------------------------------------------')
    f.close()

#根据局部原子坐标，找到训练出来的npy匹配的真实的文件名
#path：真实文件的存放路径
#path_test：训练出来需要更改为真实文件名的npy文件
def change_filename(path,path_test): 
    for filename in os.listdir(path):
        path_angle = os.path.join(path,filename)
        load = np.array(np.load(path_angle))
        for file in os.listdir(path_test):
            path_fake = os.path.join(path_test,file)
            load_fake = np.array(np.load(path_fake))
            if np.linalg.norm(load[0][0:3]-load_fake[2][0:3]) == 0:
                print(filename)
                path_new = os.path.join(path_test,filename)
                os.rename(path_fake,path_new)             
            
# path = 'G:\\Computational reconstruction\\plane_torsion'
# path_test =  'G:\\Computational reconstruction\\torsion_validation'
# change_filename(path,path_test)


def traverse_file(path):
    ask = input('是否要计算旋转角？ y/n:')
    ask_repetition = input('是否采用05-文献方法计算坐标？ y/n:')
    ask_CA_CA = input('是否以CA_CA为旋转轴？ y/n:')
    ask_intersection = input('是否通过求两个环的交点预测原子坐标？ y/n:')
    
    for i in os.listdir(path):
        path_file = os.path.join(path, i)
        #替换路径为角度npy的路径（训练出来的旋转角度）
        #需和原始数据放在同一个大文件夹下 且文件名相匹配
        path_angle_ = path_file.replace('test','torsion_validation')
        path_angle = path_angle_.replace('.pn','.npy')
        
        #根据不同情况执行对应情况函数
        if ask == 'y':
            Cartesian_to_angle(path_file)
            
        elif ask_repetition == 'y':
            if i.split('.')[-1] == 'cif':
                angle_to_Cartesian_cif(path_file, path_angle)
            elif i.split('.')[-1] == 'pn':
                angle_to_Cartesian(path_file, path_angle)
                
        elif ask_CA_CA == 'y':
            angle_to_Cartesian_CA_CA(path_file, path_angle)
            
        elif ask_intersection == 'y':   
            angle_to_Cartesian_intersection(path_file,path_angle)
            

#         angle_to_Cartesian_CA_C(path_cif,path_angle)#计算CA-C为轴长的复原方法
#         angle_to_Cartesian_CA_CA(path_cif,path_angle)#3-CA为旋转基准面的复原方法

traverse_file('G:\Computational reconstruction\\plam\\test' )